
# In the name of God the Beneficent the Merciful
## Team: Overfit


The following report is organized in this order first we go over our best reuslt and then 

we would mention our unsuccessful attempts

## 1- Pipeline
## 2- Preprocessing
* Timestamp
* Page Topic
* one-hot encoding


## 3- Modeling
* Logistic Regression
* Evaluation
* Submit


## 4- Unsuccessful attempts
* Bayesian Probablity Method with joint pair wise probability
* Label Smoothing
* Factorization Machine
* Wide and Deep

# 1- **Pipeline**


![title](img/pipeline.jpg)

# 2- **Preprocessing**

In [0]:
import pandas as pd

In [5]:
!unzip drive/My\ Drive/datadays2020_contest_public_dataset.zip 

In [0]:
df_ad = pd.read_csv('datadays2020_contest_public_dataset/ad.csv')
df_event = pd.read_csv('datadays2020_contest_public_dataset/event.csv')
df_page = pd.read_csv('datadays2020_contest_public_dataset/page.csv')
df_topic = pd.read_csv('datadays2020_contest_public_dataset/page_topic.csv')
df_train = pd.read_csv('datadays2020_contest_public_dataset/click_train.csv')
df_test = pd.read_csv('datadays2020_contest_public_dataset/click_test.csv')

## Timestamp

1. We shifted the timestamp by first timestamp.
2. Then we took modulus 24 (hours), and categorized it in  4 seqment of day.
  

In [0]:
t=min(df_event['timestamp'])
timetable = [(0,6), (6,12),(12,18),(18,24)]
def discretiseTime(t,timetable):
    t=t%24
    for i , e in enumerate(timetable):
        #print(t , e)
        if t>=e[0] and t<= e[1]:
            return i

df['timestamp'] = df['timestamp'].apply(lambda x : discretiseTime((x-t)/(1000*3600) , timetable)) 
dft['timestamp'] = dft['timestamp'].apply(lambda x : discretiseTime((x-t)/(1000*3600) , timetable))                                                 
df.head()

**Page Topic**

first we have understood that number of pagetopics with acceptable confidence is large so we discard the confidence entirely 

In [0]:
x = len(df_pageTopic['pageId'].unique())
a = len(df_pageTopic.loc[df_pageTopic['confidence'] > 0.5]) # with bigger than 50 percent probablity
b = len(df_pageTopic.loc[df_pageTopic['confidence'] > 0.6]) # with bigger than 60 percent probablity
c = len(df_pageTopic.loc[df_pageTopic['confidence'] > 0.8]) # with bigger than 80 percent probablity

print("We have estimation for {} pageId".format(c))
print("For 0.5 confidence, the ratio is {}".format(a/x))
print("For 0.6 confidence, the ratio is {}".format(b/x))
print("For 0.8 confidence, the ratio is {}".format(c/x))

So we just kept topicId with maximum confidence value.

In [0]:
df_topic=df_topic.sort_values(['pageId' , 'confidence'] , ascending=False)
df_topic=df_topic.drop_duplicates(['pageId'])

**Merge**

**df** := result of merging all dataframe except words and image with train data

**dft** := result of merging all dataframe except words and image with test data

In [0]:
df = pd.merge(df_train , df_event , on='displayId' , how='left')
dft = pd.merge(df_test , df_event , on='displayId' , how='left')
df = pd.merge(df , df_page , on='pageId' , how='left')
dft = pd.merge(dft , df_page , on='pageId' , how='left')
df_topic=df_topic.sort_values(['pageId' , 'confidence'] , ascending=False)
df_topic=df_topic.drop_duplicates(['pageId'])
df = pd.merge(df , df_topic , on='pageId' , how='left')
dft = pd.merge(dft , df_topic , on='pageId' , how='left')
df = pd.merge(df , df_ad , on='adId' , how='left')
dft = pd.merge(dft , df_ad , on='adId' , how='left')

**Fill null value**

In [0]:
df['topicId'] = df['topicId'].fillna(0)
df['confidence'] = df['confidence'].fillna(0)
dft['topicId'] = dft['topicId'].fillna(0)
dft['confidence'] = dft['confidence'].fillna(0)

**Changing type for better speed and RAM usage** 

In [0]:
df['clicked'] = df['clicked'].astype(np.int8)
df['timestamp'] = df['timestamp'].astype(np.int8)
df['pageId'] = df['pageId'].astype(np.int32)
df['widgetId'] = df['widgetId'].astype(np.int16)
df['userId'] = df['userId'].astype(np.int32)
df['device'] = df['device'].astype(np.int8)
df['OS'] = df['OS'].astype(np.int8)
df['browser'] = df['browser'].astype(np.int8)
df['website'] = df['website'].astype(np.int32)
df['publisher'] = df['publisher'].astype(np.int16)
df['topicId']= df['topicId'].astype(np.int8)
df['confidence'] =df['confidence'].astype(np.float32)
df['campaignId'] = df['campaignId'].astype(np.int16)
df['advertiserId'] = df['advertiserId'].astype(np.int16)

#dft['clicked'] = dft['clicked'].astype(np.int8)
dft['timestamp'] = dft['timestamp'].astype(np.int8)
dft['pageId'] = dft['pageId'].astype(np.int32)
dft['widgetId'] = dft['widgetId'].astype(np.int16)
dft['userId'] = dft['userId'].astype(np.int32)
dft['device'] = dft['device'].astype(np.int8)
dft['OS'] = dft['OS'].astype(np.int8)
dft['browser'] = dft['browser'].astype(np.int8)
dft['website'] = dft['website'].astype(np.int32)
dft['publisher'] = dft['publisher'].astype(np.int16)
dft['topicId']= dft['topicId'].astype(np.int8)
dft['confidence'] =dft['confidence'].astype(np.float32)
dft['campaignId'] = dft['campaignId'].astype(np.int16)
dft['advertiserId'] = dft['advertiserId'].astype(np.int16)

Features (12 columns) for first logistic regression 

(intiution was it would capture a general notion of importance on all features and can be used as base estimate)

In [0]:
features = ['userId','adId' , 'timestamp' , 'device' , 'browser' , 'OS' , 'topicId'   , 'campaignId' , 'advertiserId' , 'widgetId' , 'website' , 'publisher']

Features1 is less inclusive to distinguish between different ads

In [0]:
features1 = ['adId'  , 'topicId'   , 'campaignId' , 'advertiserId' , 'widgetId']

This heat map shows relative importance of different features pair-wise 

which persuade us choose above features as our distinguishing factors between ads 

![title](img/heatmap.PNG)

In [0]:
dftemp = df[features]
dfttemp = dft[features]
dftemp.head()

In [0]:
dftemp1 = df[features1]
dfttemp1 = dft[features1]
dftemp1.head()

**Spliting test and train data**

We define double test and train because of using ensemble method in our model that bring after in modelling part 

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train , X_test , Y_train , Y_test = train_test_split(dftemp , df['clicked'] , test_size = 0.33 , random_state = 42)

In [0]:
X_train1 , X_test1 , Y_train1 , Y_test1 = train_test_split(dftemp1 , df['clicked'] , test_size = 0.33 , random_state = 42)

Spliting data in equal portion for those algorithms without a balancing feature built in 

because of class imbalance problem


In [2]:
#it should only be run for algorithms without built in balacing
n = int(0.66*len(df))
tplus = df[df['clicked']>0].sample(n=n//2 , replace=True)
tminus = df[df['clicked']==0].sample(n=n//2 , replace=True)
X_train = pd.concat([tplus ,tminus])
X_train = X_train.sample(frac=1.00)
Y_train = X_train['clicked']
X_train = X_train[features]

## **One-Hot Encoding**

because of categorical data, we need to use one-hot encoder from sklearn library. and keep then **csr matrix**


In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
transformer = ColumnTransformer(  
    transformers=[
        ("all",        # Just a name
         OneHotEncoder(dtype = np.int8), # The transformer class
         [0,1,2,3,4,5,6,7,8,9,10,11]           # The column(s) to be applied on.
         ),
    ], remainder='passthrough'
)

transformer.fit(dftemp.append(dfttemp)) # in order to capture all categorizes we merge both train and test

In [0]:
data = transformer.transform(X_train)
data.shape

In [0]:
data = transformer.transform(X_train1)
data.shape

# 3- **Modeling**

Our model is based on Logistic Regression method for classification of our data.



In [0]:
from sklearn.linear_model import LogisticRegression 

## Logistic Regression
**The Logistic Regression parameter** 

C is the reverse of lambda value in cost function. After exhusive seaech on **(0.6,1)** interval we found **0.72** as optimal value

Class_weight is **balanced** because of the fact that 77 percent of the value is 0 and would push the algorithm to produce zeros and would weight columns with more zeros in them which is not desirable to encounter this;we balance the classes

In [0]:
clf = LogisticRegression(class_weight='balanced' , C=0.72).fit(data,Y_train)

In [0]:
clf5 = LogisticRegression(class_weight='balanced' , C=0.72).fit(data,Y_train1)

In [0]:
Xtest = dft
datates = transformer.transform(dfttemp)

In [0]:
Xtest1 = dft1 
datates1 = transformer.transform(dfttemp1)

## **Evaluation**

In [0]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [0]:
clf.score(data,Y_train)

In [0]:
clf5.score(data,Y_train1)

## **Creating submit dataset**

In [0]:
Xtest['rank'] = clf.predict_proba(datates)[:,1] + clf5.predict_proba(datates1)[:,1] #ensembling two models

In [0]:
diad={}
for i,r in Xtest.iterrows():
    if r['displayId'] in diad:
        diad[r['displayId']].append(i)
    else:
        diad[r['displayId']] = [i,]
for i in diad:
    diad[i] = sorted(diad[i] , key =lambda x: -1 * Xtest.iloc[x]['rank'] )
arr = np.zeros(len(Xtest))

for i,r in Xtest.iterrows():
    arr[i] = diad[r['displayId']].index(i) + 1
Xtest['ranked'] = arr
#dftest.to_csv('../submit/sub4.csv',header=None,index=False)
dfsub = Xtest[['displayId' , 'adId' ,'ranked']]
dfsub['ranked'] = Xtest['ranked'].apply(lambda x : int(x) )
dfsub.head()
dfsub.to_csv('Sub29.csv',header=None,index=False)

# 4- **Unsuccessful attempts**

## Bayesian Probablity Method with joint pair wise probability

In [ ]:
anchors = ['adId' , 'campaignId' , 'advertiserId']
feature = ['pageId','userId','widgetId' , 'topicId' ,'website' , 'timestamp']
features = [(anchor , f) for anchor in anchors for f in feature]
column_names = anchors + features

In [0]:
def create_real_train4(anchors , features,column_names,x_train ):
    count=0;
    n = len(x_train)
    print(n)
    dfdummy = pd.DataFrame(columns = column_names)
    featuregroups = {}
    for anchor in anchors:
        featuregroups[anchor] = dftrain.groupby(anchor).mean()['clicked']
    for f in features:
        featuregroups[f] = dftrain.groupby(f).mean()['clicked']
    temps=[]
    for i,r in x_train.iterrows():
        count+=1
        if(count%100000==0):
            print("--{}--".format(count/n))
        temp=[]
        for anchor in anchors:
            tr = featuregroups[anchor].get(r[anchor])
            temp.append(tr if tr!=None else 0)
        for f in features:
            tr = featuregroups[f].get((r[f[0]] , r[f[1]]))
            temp.append(tr if tr!=None else 0)
        temps.append(temp)
    dfdummy=dfdummy.append(pd.DataFrame(temps, columns=column_names) , ignore_index=True)
    return dfdummy

In [0]:
X = create_real_train(features[:] , column_names , X_train, word_prob=probs)

In [0]:

clf = RandomForestClassifier(class_weight = 'balanced').fit(X , y_train['clicked'])

## Label Smoothing

using label smoothing to change classification to regression which showed enhancing results in competitions like image net

[Rethinking the Inception Architecture for Computer Vision](https://arxiv.org/abs/1512.00567)

In [ ]:
ul=[]
def add_to(x):
    ul.append(x)
    return 1
temp=dftrainY['displayId'].value_counts()
temp.index=temp.index.astype(str)
td = temp.to_dict()
dftrainY['displayId'] = dftrainY['displayId'].astype(str)

#df['displayId'] = df['displayId'].astype(int)

dftrainY['Frecuency'] = dftrainY['displayId'].apply(lambda x : td.get(x) if td.get(x)!=None else add_to(x))


dftrainY["probability"] = (dftrainY["clicked"]+1)/(dftrainY["Frecuency"]*2)

#print(df.head(5))

## Factorization Machine

Factorization Machines
(FM) which are a new model class that combines the advantages
of Support Vector Machines (SVM) with factorization models
thus
they are able to estimate interactions even in problems with huge
sparsity (like recommender systems) where SVMs fail.

[Factorization Machines Steffen Rendle 2010](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)

[Field-aware Factorization Machines for CTR Prediction](https://www.csie.ntu.edu.tw/~cjlin/papers/ffm.pdf)

In [0]:
class FactorizationMachine(nn.Module):
    def __init__(self, nb_features, dim_embed=50, isClassifier=True, withCuda=True):
        super(FactorizationMachine, self).__init__()
        
        self.nb_features = nb_features
        self.dim_embed = dim_embed
        self.isClassifier = isClassifier # binary-classifier or regression
        
        # Stores the bias term
        if withCuda: # WARNING : not working at present with CUDA due to type mismatch
            self.B = Variable(torch.randn((1)).type(torch.cuda.FloatTensor), requires_grad=True)
        else:
            self.B = Variable(torch.randn((1)).type(torch.FloatTensor), requires_grad=True)
        
        # Stores the weights for the linear terms
        self.embeddingL = nn.Embedding(nb_features, 1, padding_idx=0, max_norm=None, norm_type=2)
        
        # Stores the weights for the quadratic FM terms
        self.embeddingQ = nn.Embedding(nb_features, dim_embed, padding_idx=0, max_norm=None, norm_type=2)

        
    def forward(self, X):
        
        # The linear part
        eL = self.embeddingL(X)
        logitL = eL.sum(dim=1)
        
        # The Quadratic-FM part using the O(kn) formulation from Steffen Rendle
        eQ = self.embeddingQ(X)
        logitFM1 = eQ.mul(eQ).sum(1).sum(2)
        z = eQ.sum(dim=1)# sum across features
        z2 = z.mul(z) # element-wise product
        logitFM2 = z2.sum(dim=2) # sum across embedding dimensions
        logitFM = (logitFM1 - logitFM2)*0.5
        
        # Total logit
        logit = (logitL + logitFM).squeeze(dim=-1).squeeze(dim=-1)
        logit+= self.B.expand(1, logit.size()[0]).transpose(0,1)
        
        if self.isClassifier:
            return F.sigmoid(logit)
        else:
            return logit

In [0]:
model = FactorizationMachine(100, dim_embed=50, isClassifier=True, withCuda=use_cuda)

In [0]:
learning_rate = 0.1
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

#loss_function = nn.NLLLoss()
loss_function = nn.CrossEntropyLoss()

Or using [tffm](https://github.com/geffy/tffm)
package an open source package implementing factorization Machine

In [4]:
import tensorflow as tf
from tffm import TFFMClassifier 
order = 3
model = TFFMClassifier(
    order=order, 
    rank=8, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=50, 
    batch_size=512,
    init_std=0.001,
    reg=0.001,
    input_type='sparse',
    seed=42
)
model.fit(data, Y_train.values, show_progress=True)
predictions = model.predict(data)
print('[order={}] accuracy: {}'.format(order, accuracy_score(Y_train.values, predictions)))
predictions = model.predict(datatest)
print('[order={}] accuracy: {}'.format(order, accuracy_score(Y_test.values, predictions)))
#model.destroy()

## Wide and Deep 

Wide & Deep learning—jointly trained wide linear models
and deep neural networks—to combine the benefits of memorization and generalization for recommender systems

[Wide and Deep Learning for Recommender Systems 2016](https://arxiv.org/abs/1606.07792)

In [0]:
from sklearn.metrics import roc_auc_score
from keras.models import Sequential , Model
from keras.layers import Input , Dense , concatenate

#wide
wide = Input(shape=(data.shape[1],))

#deep
deep_data = Input(shape=(data.shape[1],))
deep = Dense(input_dim=data.shape[1] , output_dim=256 , activation='relu')(deep_data)
deep = Dense(128 , activation = 'relu')(deep)

#wide & deep
wide_deep = concatenate([wide , deep])
wide_deep = Dense(1 , activation='sigmoid')(wide_deep)
model = Model(inputs=[wide , deep_data] , outputs=wide_deep)

model.compile(
    optimizer = 'rmsprop' ,
    loss = 'binary_crossentropy' ,
    metrics = ['accuracy']
)


In [0]:
from keras.models import load_model
model = load_model('../my_model.h5') #if there were any prior training

In [0]:
model.fit([data , data] , Y_train , epochs=5 , batch_size=512)